In [1]:
na = (In=1:9, Fl=1.0:9)

(In = 1:9, Fl = 1.0:1.0:9.0)

In [2]:
# At compile time, the compiler does not know that the `Symbol` is `:Char`.
# It compiles getproperty(ta, ::Symbol), which returns `Vector{Any}`.
@code_warnaype na.In

MethodInstance for getproperty(::NamedTuple{(:In, :Fl), Tuple{UnitRange{Int64}, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}}, ::Symbol)
  from getproperty(x, f::Symbol) in Base at Base.jl:38
Arguments
  #self#::Core.Const(getproperty)
  x::NamedTuple{(:In, :Fl), Tuple{UnitRange{Int64}, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}}
  f::Symbol
Body::Union{StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}, UnitRange{Int64}}
1 ─      nothing
│   %2 = Base.getfield(x, f)::Union{StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}, UnitRange{Int64}}
└──      return %2



In [3]:
# At compile time, the compiler knows that the `Symbol` is `:Char`.
# It compiles getproperty(ta, :Char), which returns `Vector{Char}`.
@code_warnaype (na->na.In)(na)

MethodInstance for (::var"#1#2")(::NamedTuple{(:In, :Fl), Tuple{UnitRange{Int64}, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}})
  from (::var"#1#2")(na) in Main at In[3]:3
Arguments
  #self#::Core.Const(var"#1#2"())
  na::NamedTuple{(:In, :Fl), Tuple{UnitRange{Int64}, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}}
Body::UnitRange{Int64}
1 ─ %1 = Base.getproperty(na, :In)::UnitRange{Int64}
└──      return %1



If the compiler knows the columns extracted from a `NamedTuple` at compile time, the compiler optimizes code and return type.

In [9]:
na = (ab=[1, 2], cd=[3.0, 4.0], ef=[true, false])

luffy(na, id) = na[id]

dragon(na) = luffy(na, 1)

;

In [10]:
@code_warntype luffy(na, 1)

MethodInstance for luffy(::NamedTuple{(:ab, :cd, :ef), Tuple{Vector{Int64}, Vector{Float64}, Vector{Bool}}}, ::Int64)
  from luffy(na, id) in Main at In[9]:3
Arguments
  #self#::Core.Const(luffy)
  na::NamedTuple{(:ab, :cd, :ef), Tuple{Vector{Int64}, Vector{Float64}, Vector{Bool}}}
  id::Int64
Body::Union{Vector{Bool}, Vector{Float64}, Vector{Int64}}
1 ─ %1 = Base.getindex(na, id)::Union{Vector{Bool}, Vector{Float64}, Vector{Int64}}
└──      return %1



In [ ]:
@code_warntype dragon(na)

It is not enough to have a type-stable data structure but also type-stable code.